In [10]:
import session_info

import itertools
import os
import re
import shutil
import sys
from pathlib import Path
from typing import Iterable, Literal

import anndata as ad
import numpy as np
import scipy.sparse as sp
import duckdb
import pandas as pd
import requests
from appdirs import user_cache_dir
from tqdm import tqdm

In [27]:
REMOTE_URL = "https://object-store.rc.nectar.org.au/v1/AUTH_06d6e008e3e642da99d806ba3ea629c5"
ASSAY_URL = "{}/cellNexus-anndata".format(REMOTE_URL)
METADATA_URL = "{}/cellNexus-metadata/metadata.1.0.11.parquet".format(REMOTE_URL)
MIN_EXPECTED_SIZE = 5000000

assay_map = {"counts": "counts", "cpm": "cpm"}

def is_parquet_valid(parquet_file):
    try:
        conn = duckdb.connect()
        conn.from_parquet(str(parquet_file))  # Try reading
        return True  # File is valid
    except Exception as e:
        print(f"Parquet file is corrupt: {e}")
        return False  # File is corrupt
        
def _get_default_cache_dir() -> Path:
    return Path(user_cache_dir("cellNexusPy"))


# helper function to download file over http/https
def _sync_remote_file(full_url: str, output_file: Path):
    if not output_file.exists():
        output_dir = output_file.parent
        output_dir.mkdir(parents=True, exist_ok=True)
        print(f"Downloading {full_url} to {output_file}", file=sys.stderr)
        req = requests.get(full_url, stream=True, allow_redirects=True)
        req.raise_for_status()
        pbar = tqdm(total=int(req.headers.get("Content-Length", 0)))
        with pbar.wrapattr(req.raw, "read") as src, output_file.open("wb") as dest:
            shutil.copyfileobj(src, dest)

# function to get metadata
def get_metadata(
    parquet_url: str = METADATA_URL,
    cache_dir: os.PathLike[str] = _get_default_cache_dir(),
) -> tuple[duckdb.DuckDBPyConnection, duckdb.DuckDBPyRelation]:
    parquet_local = Path(cache_dir) / parquet_url.split("/")[-1]

    if not parquet_local.exists() or not is_parquet_valid(parquet_local):
        print("File is missing or corrupted. Re-downloading...")
        parquet_local.unlink(missing_ok=True)  # Delete the corrupted file
        _sync_remote_file(parquet_url, parquet_local)  # Re-download
    
    _sync_remote_file(parquet_url, parquet_local)
    conn = duckdb.connect()
    return conn, conn.from_parquet(str(parquet_local))

def sync_assay_files(
    url: str = ASSAY_URL,
    cache_dir: Path = _get_default_cache_dir(),
    subdirs: Iterable[str] = [],
    atlases: Iterable[str] = [],
    aggregation: Iterable[str] = [],
    files: Iterable[str] = [],
):
    for atlas in atlases:
        for subdir in subdirs:
            for file in files:
                if aggregation == "single_cell":
                    sub_url = f"{url}/{atlas}/{subdir}/{file}"
                else:
                    sub_url = f"{url}/{atlas}/{aggregation}/{subdir}/{file}"
                output_filepath = cache_dir / atlas / aggregation / subdir / file

                if not output_filepath.exists() or os.path.getsize(output_filepath) < MIN_EXPECTED_SIZE:
                    _sync_remote_file(sub_url, output_filepath)

                yield subdir, output_filepath

In [113]:
def _filter_obs(data,aggregation,anndata,cells,features):
    if aggregation == "single_cell":
        anndata.obs.index = anndata.obs.index.astype(str)
        cell_ids = cells.astype(str)
        pattern = '|'.join(re.escape(s) for s in cell_ids)
        mask = anndata.obs.index.str.contains(pattern, regex=True)
    
        anndata = anndata[mask,features].copy()
        
        positions_per_cell = []
        
        for cid in cell_ids:
            pos = np.where(anndata.obs.index.str.contains(cid))[0]
            positions_per_cell.append(pos)
    
        ann = anndata[np.concatenate(positions_per_cell).tolist(),:].copy()
        ann.obs = data.fetchdf()
        ann.obs.index = ann.obs["cell_id"]
        
    elif aggregation == "pseudobulk":
        cell_ids = data.project('"sample_id"').fetchdf()["sample_id"]+"___"+data.project('"cell_type_unified_ensemble"').fetchdf()["cell_type_unified_ensemble"]
        ann = anndata[cell_ids.unique(),features].copy()

        columns_to_remove = ["cell_id", "cell_type", "file_id_cellNexus_single_cell",
                         "cell_type_ontology_term_id",
                         "observation_joinid", "ensemble_joinid",
                         "nFeature_RNA", "data_driven_ensemble", "cell_type_unified",
                         "empty_droplet", "observation_originalid", "alive", "scDblFinder.class"]
        subdata = data.fetchdf().drop(columns=[col for col in columns_to_remove if col in data.fetchdf()])
        pattern = '|'.join(re.escape(s) for s in ["metacell","azimuth","monaco","blueprint","subsets_","high_"])
        
        # Find matching columns and drop them
        cols_to_drop = subdata.columns[subdata.columns.str.contains(pattern, case=False, regex=True)]
        subdata = subdata.drop(columns=cols_to_drop)
        subdata = subdata.drop_duplicates(keep='last')
        subdata.index = subdata["sample_id"]+"___"+subdata["cell_type_unified_ensemble"]

        ann.obs = subdata.reindex(ann.obs.index)
    return ann

def get_anndata(
    data: duckdb.DuckDBPyRelation,
    assays: Iterable[Literal["counts", "cpm"]] = ["counts", "cpm"],
    aggregation: Iterable[Literal["single_cell", "pseudobulk", "metacell"]] = ["single_cell", "pseudobulk", "metacell"],
    cache_directory: Path = _get_default_cache_dir(),
    repository: str = ASSAY_URL,
    features: Iterable[str] = slice(None, None, None),
) -> ad.AnnData:
    # error checking
    assert set(assays).issubset(set(assay_map.keys()))
    assert isinstance(cache_directory, Path), "cache_directory must be a Path"
    
    assays = set(assays)
    cache_directory.mkdir(exist_ok=True, parents=True)
    
        # all the files to retrieve for query
    if str(aggregation[0])=="metacell":
        aggregation = ["single_cell"] 
        data = data.filter("metacell_2 IS NOT NULL")
    
    files_to_read = (
        data.project("file_id_cellNexus_"+str(aggregation[0])).distinct().fetchdf()["file_id_cellNexus_"+str(aggregation[0])]
    )
    cells = data.project('"cell_id"').distinct().fetchdf()["cell_id"]
    atlases = data.project('"atlas_id"').distinct().fetchdf()["atlas_id"]
        # mapping requested assay to subdirectory name
    subdirs = [assay_map[a] for a in assays]                                                                                                                         
    
    synced = sync_assay_files(
        url=repository, cache_dir=cache_directory, atlases=atlases, subdirs=subdirs, aggregation=aggregation[0], files=files_to_read
    )
    
    ad_per_assay: dict[str, ad.AnnData] = {}
        # Concatenate files per assay
    for assay_name, files in itertools.groupby(synced, key=lambda x: x[0]):
        ads = [ad.read_h5ad(file[1]) for file in files]
        ad_per_assay[assay_name] = ad.concat(ads)
    
    main_assay_key = (
        "counts" if "counts" in ad_per_assay else next(iter(ad_per_assay.keys()))
    )
    
    ann = ad_per_assay.pop(main_assay_key)
        # Attach the additional assays
    for key, anndata in ad_per_assay.items():
        ann.layers[key] = anndata.X
    
    ann = _filter_obs(data,aggregation[0],ann,cells,features)
    return ann

In [6]:
session_info.show()

/home/juan/anaconda3/envs/cellnexuspy/lib/python3.10/site-packages/session_info/main.py:213: UserWarning: The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.
  mod_version = _find_version(mod.__version__)


In [ ]:
conn, table = get_metadata()
table

File is missing or corrupted. Re-downloading...


  0%|                                                                                                                                                | 0/2971497736 [00:00<?, ?it/s]
0it [00:00, ?it/s]
64.0kB [00:00, 111kB/s]
128kB [00:00, 158kB/s] 
192kB [00:01, 179kB/s]
448kB [00:01, 418kB/s]
896kB [00:01, 778kB/s]
1.56MB [00:02, 886kB/s]
2.88MB [00:02, 1.94MB/s]
3.50MB [00:02, 2.20MB/s]
3.94MB [00:03, 1.84MB/s]
4.19MB [00:03, 1.68MB/s]
5.44MB [00:03, 2.50MB/s]
6.19MB [00:03, 3.14MB/s]
6.62MB [00:04, 2.76MB/s]
7.06MB [00:04, 2.41MB/s]
7.88MB [00:04, 2.54MB/s]
8.69MB [00:04, 3.33MB/s]
9.12MB [00:05, 2.85MB/s]
9.62MB [00:05, 2.58MB/s]
10.4MB [00:05, 3.38MB/s]
10.8MB [00:05, 2.87MB/s]
11.4MB [00:05, 2.71MB/s]
12.1MB [00:06, 3.44MB/s]
12.5MB [00:06, 2.90MB/s]
13.2MB [00:06, 2.70MB/s]
14.0MB [00:06, 3.61MB/s]
14.5MB [00:06, 3.07MB/s]
15.1MB [00:07, 2.78MB/s]
15.9MB [00:07, 3.67MB/s]
16.4MB [00:07, 3.12MB/s]
16.9MB [00:07, 2.90MB/s]
17.5MB [00:07, 3.37MB/s]
17.9MB [00:08, 3.03MB/s]
18.5MB [0

In [14]:
table.aggregate("tissue, cell_type_unified_ensemble, COUNT(*) as n", group_expr="tissue, cell_type_unified_ensemble")

┌─────────────────────────────────┬────────────────────────────┬───────┐
│             tissue              │ cell_type_unified_ensemble │   n   │
│             varchar             │          varchar           │ int64 │
├─────────────────────────────────┼────────────────────────────┼───────┤
│ lung                            │ nk                         │ 91678 │
│ respiratory airway              │ pneumocyte                 │  9282 │
│ lung                            │ cd4 th2 em                 │ 10537 │
│ lung parenchyma                 │ macrophage                 │ 90063 │
│ lung parenchyma                 │ cd16 mono                  │   283 │
│ myelencephalon                  │ glial                      │ 32144 │
│ myelencephalon                  │ immune                     │   238 │
│ lower lobe of left lung         │ Unknown                    │   316 │
│ lower lobe of left lung         │ epithelial                 │  3277 │
│ trachea                         │ nk             

In [15]:
query = table.filter("""
    self_reported_ethnicity = 'African'
    AND assay LIKE '%10%'
    AND tissue = 'lung parenchyma'
    AND cell_type LIKE '%CD4%'
""")
query

┌───────────────────────────────────────────────────────────────────────┬──────────────────────────────────────┬────────────────────┬──────────────────────────────────────────────────┬─────────────────────────────────┬────────────────────────────┬──────────────────────────────────┬───────────┬────────────────────────┬────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬───────────────────┬─────────────────────────┬────────────────────────────────────┬─────────┬──────────────────────────┬───────────────────────────────────────────────────────────────────────────────┬───────────────┬─────────────────────────────────────────────────────────────────────

# COUNTS EXAMPLE

In [35]:
ann = get_anndata(query, assays=["counts"],aggregation=["single_cell"])
ann

AnnData object with n_obs × n_vars = 1800 × 56239
    obs: 'cell_id', 'dataset_id', 'observation_joinid', 'sample_id', 'cell_type', 'cell_type_ontology_term_id', 'sample_', 'assay', 'assay_ontology_term_id', 'cell_count', 'citation', 'collection_id', 'dataset_version_id', 'default_embedding', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'experiment___', 'explorer_url', 'feature_count', 'filesize', 'filetype', 'is_primary_data', 'mean_genes_per_cell', 'organism', 'organism_ontology_term_id', 'primary_cell_count', 'published_at', 'raw_data_location', 'revised_at', 'run_from_cell_id', 'sample_heuristic', 'schema_version', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'title', 'tombstone', 'url', 'x_approximate_distribution', 'X_umap1', 'X_umap2', 'age_days', 'tissue_groups', 'atlas_id', 'cell_annotat

# CPM EXAMPLE

In [19]:
ann = get_anndata(query, assays=["cpm"],aggregation=["single_cell"])
ann

  0%|                                                                                                                                                   | 0/3685968 [00:00<?, ?it/s]
0it [00:00, ?it/s]
64.0kB [00:00, 95.6kB/s]
128kB [00:00, 141kB/s]  
192kB [00:01, 167kB/s]
384kB [00:01, 327kB/s]
768kB [00:01, 637kB/s]
1.25MB [00:02, 743kB/s]
2.19MB [00:02, 1.37MB/s]
2.62MB [00:03, 1.40MB/s]
3.52MB [00:03, 1.05MB/s]
  0%|                                                                                                                                                   | 0/2510302 [00:00<?, ?it/s]
0it [00:00, ?it/s]
64.0kB [00:00, 112kB/s]
128kB [00:00, 159kB/s] 
192kB [00:01, 181kB/s]
448kB [00:01, 421kB/s]
896kB [00:01, 783kB/s]
1.38MB [00:02, 977kB/s]
1.88MB [00:02, 1.19MB/s]
2.39MB [00:05, 495kB/s] 
  0%|                                                                                                                                                   | 0/3205421 [00:00<?, ?it/s]
0it [00:00

AnnData object with n_obs × n_vars = 1800 × 56239
    obs: 'cell_id', 'dataset_id', 'observation_joinid', 'sample_id', 'cell_type', 'cell_type_ontology_term_id', 'sample_', 'assay', 'assay_ontology_term_id', 'cell_count', 'citation', 'collection_id', 'dataset_version_id', 'default_embedding', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'experiment___', 'explorer_url', 'feature_count', 'filesize', 'filetype', 'is_primary_data', 'mean_genes_per_cell', 'organism', 'organism_ontology_term_id', 'primary_cell_count', 'published_at', 'raw_data_location', 'revised_at', 'run_from_cell_id', 'sample_heuristic', 'schema_version', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'title', 'tombstone', 'url', 'x_approximate_distribution', 'X_umap1', 'X_umap2', 'age_days', 'tissue_groups', 'atlas_id', 'cell_annotat

# PSEUDOBULK EXAMPLE

In [114]:
ann = get_anndata(query,assays=["counts"],aggregation=["pseudobulk"])
ann

AnnData object with n_obs × n_vars = 100 × 56239
    obs: 'dataset_id', 'sample_id', 'sample_', 'assay', 'assay_ontology_term_id', 'cell_count', 'citation', 'collection_id', 'dataset_version_id', 'default_embedding', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'experiment___', 'explorer_url', 'feature_count', 'filesize', 'filetype', 'is_primary_data', 'mean_genes_per_cell', 'organism', 'organism_ontology_term_id', 'primary_cell_count', 'published_at', 'raw_data_location', 'revised_at', 'run_from_cell_id', 'sample_heuristic', 'schema_version', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'title', 'tombstone', 'url', 'x_approximate_distribution', 'X_umap1', 'X_umap2', 'age_days', 'tissue_groups', 'atlas_id', 'cell_type_unified_ensemble', 'is_immune', 'sample_chunk', 'cell_chunk', 'sample_pseudobul

# METACELL EXAMPLE

In [89]:
ann = get_anndata(query,aggregation=["metacell"])
ann

AnnData object with n_obs × n_vars = 1405 × 56239
    obs: 'cell_id', 'dataset_id', 'observation_joinid', 'sample_id', 'cell_type', 'cell_type_ontology_term_id', 'sample_', 'assay', 'assay_ontology_term_id', 'cell_count', 'citation', 'collection_id', 'dataset_version_id', 'default_embedding', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'experiment___', 'explorer_url', 'feature_count', 'filesize', 'filetype', 'is_primary_data', 'mean_genes_per_cell', 'organism', 'organism_ontology_term_id', 'primary_cell_count', 'published_at', 'raw_data_location', 'revised_at', 'run_from_cell_id', 'sample_heuristic', 'schema_version', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'title', 'tombstone', 'url', 'x_approximate_distribution', 'X_umap1', 'X_umap2', 'age_days', 'tissue_groups', 'atlas_id', 'cell_annotat